In [1]:
import sqlite3
from sqlite3 import Error
import csv

# Task 1

In [ ]:
def connect_db(db_file):
    """
    Establish a connection to the SQLite database.

    Args:
        db_file (str): The database file name.

    Returns:
        Connection object or None: Connection to the database.
    """
    try:
        conn = sqlite3.connect(db_file)
        print("Database connected.")
        return conn
    except Error as e:
        print(f"Error: {e}")
        return None

In [ ]:
db_conn = connect_db("santas_workshop.db")
db_conn.close()
print("Database closed")

# Task 2

In [ ]:
def create_tables(conn):
    """
    Create 'toys' and 'recipients' tables in the database.

    Args:
        conn (Connection): The database connection object.
    """
    toys_sql = """CREATE TABLE IF NOT EXISTS `toys` (
        `toy_id` TEXT NOT NULL UNIQUE,
        `toy_type` TEXT NOT NULL,
        `toy_name` TEXT NOT NULL,
        PRIMARY KEY(`toy_id`)
    );"""

    recipients_sql = """CREATE TABLE IF NOT EXISTS `recipients` (
        `recipient_id` INTEGER NOT NULL UNIQUE,
        `recipient_toy_id` INTEGER NOT NULL UNIQUE,
        `recipient_name` TEXT NOT NULL,
        PRIMARY KEY(`recipient_id`),
        FOREIGN KEY(`recipient_toy_id`) REFERENCES `toys`(`toy_id`)
    );"""

    tables = [("toys", toys_sql), ("recipients", recipients_sql)]

    for table in tables:
        try:
            conn.execute(table[1])
            print(f"Table '{table[0]}' created successfully.")
        except Error as e:
            print(f"Error creating table '{table[0]}': {e}")

In [ ]:
db_conn = connect_db("santas_workshop.db")
create_tables(db_conn)
db_conn.close()
print("Database closed")

# Task 3

In [ ]:
def import_toys(conn, csvfilename):
    """
    Import toy data from a CSV file into the 'toys' table.

    Args:
        conn (Connection): The database connection object.
        csvfilename (str): The path to the CSV file.
    """
    sql = "INSERT INTO toys VALUES (?,?,?)"
    with open(csvfilename, "r") as csvfile:
        csvreader = csv.reader(csvfile)
        header = next(csvreader)  # Skip the header row
        count = 0
        for toy in csvreader:
            try:
                conn.execute(sql, tuple(toy))
                conn.commit()
                count += 1
            except Error as e:
                print(f"Error inserting toy: {e}")
    print(f"{count} toys imported.")

In [ ]:
db_conn = connect_db("santas_workshop.db")
import_toys(db_conn, "new_toys.csv")
db_conn.close()
print("Database closed")

# Task 4

In [ ]:
def import_recipients(conn, csvfilename):
    """
    Import recipient data from a CSV file into the 'recipients' table.

    Args:
        conn (Connection): The database connection object.
        csvfilename (str): The path to the CSV file.
    """
    sql = "INSERT INTO recipients VALUES (?,?,?)"
    with open(csvfilename, "r") as csvfile:
        csvreader = csv.reader(csvfile)
        header = next(csvreader)  # Skip the header row
        count = 0
        for person in csvreader:
            try:
                conn.execute(sql, tuple(person))
                conn.commit()
                count += 1
            except Error as e:
                print(f"Error inserting recipient: {e}")
    print(f"{count} recipients imported.")

In [ ]:
db_conn = connect_db("santas_workshop.db")
import_recipients(db_conn, "new_recipients.csv")
db_conn.close()
print("Database closed")

# Task 5

In [ ]:
def filter_contaminants(conn):
    """
    Retrieve toys with contaminant types (e.g., 'Gun', 'Bomb', 'Organ').

    Args:
        conn (Connection): The database connection object.

    Returns:
        list: A list of contaminated toys.
    """
    contaminant_types = ["Gun", "Bomb", "Organ"]
    contaminants_list = []
    try:
        for toy_type in contaminant_types:
            sql = f"SELECT toy_id, toy_name FROM toys WHERE toy_type = '{toy_type}'"
            cur = conn.execute(sql)
            rows = cur.fetchall()
            contaminants_list.extend(rows)
        return contaminants_list
    except Error as e:
        print(f"Error filtering contaminants: {e}")
        return []

In [ ]:
db_conn = connect_db("santas_workshop.db")
print(filter_contaminants(db_conn))
db_conn.close()
print("Database closed")

# Task 6

In [ ]:
def filter_nonsense(conn):
    """
    Retrieve recipients requesting toys flagged as contaminants.

    Args:
        conn (Connection): The database connection object.

    Returns:
        list: A list of recipient names.
    """
    toy_ids = [
        "34148576",
        "72953028",
        "26080049",
        "27557990",
        "15562806",
        "97319069",
        "38216491",
        "37728318",
    ]
    toy_ids_str = ", ".join([f'"{id}"' for id in toy_ids])
    sql = f"SELECT recipient_name FROM recipients WHERE recipient_toy_id IN ({toy_ids_str})"
    try:
        cur = conn.execute(sql)
        return cur.fetchall()
    except Error as e:
        print(f"Error filtering recipients: {e}")
        return []

In [ ]:
db_conn = connect_db("santas_workshop.db")
print(filter_nonsense(db_conn))
db_conn.close()
print("Database closed")

# Tasks 7-9

In [ ]:
# Task 7
def delete_person(conn):
    sql = "DELETE FROM recipients WHERE recipient_name IN ('Luocha', 'Klee', 'Aventurine', 'MrReca', 'Joaqium', 'Blade', 'Kafka', 'Mobius')"
    try:
        conn.execute(sql)
        conn.commit()
        print("Specific recipients deleted successfully.")
    except Error as e:
        print(f"Error deleting recipients: {e}")


# Task 8
def delete_threat(conn):
    toy_ids = [
        "34148576",
        "72953028",
        "26080049",
        "27557990",
        "15562806",
        "97319069",
        "38216491",
        "37728318",
    ]
    toy_ids_str = ", ".join([f'"{id}"' for id in toy_ids])
    sql = f"DELETE FROM toys WHERE toy_id IN ({toy_ids_str})"
    try:
        conn.execute(sql)
        conn.commit()
        print("Dangerous toys deleted successfully.")
    except Error as e:
        print(f"Error deleting toys: {e}")


# Task 9
def join_tables(conn):
    sql = """SELECT toys.toy_type, toys.toy_name, recipients.recipient_id, recipients.recipient_name
             FROM toys
             JOIN recipients ON toys.toy_id = recipients.recipient_toy_id;"""
    try:
        cur = conn.execute(sql)
        return cur.fetchall()
    except Error as e:
        print(f"Error joining tables: {e}")
        return []

In [ ]:
db_conn = connect_db("santas_workshop.db")
delete_person(db_conn)
delete_threat(db_conn)
print(join_tables(db_conn))
db_conn.close()
print("Database closed")